In [ ]:
pip install langchain chromadb sentence-transformers langchain-community

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import json

In [2]:
try :
  with open("chunks.json", 'r', encoding = 'utf-8') as f :
    chunks = json.load(f)
except :
  print("error")

In [9]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document

In [10]:
data = [Document(metadata = chunk['metadata'], page_content = chunk['page_content']) for chunk in chunks]
len(data)

2729

In [11]:
# Option 1: A standard, fast, and excellent baseline
# model_name = "sentence-transformers/all-MiniLM-L6-v2"

# Option 2: DPR (Dense Passage Retrieval)
# model_name = "facebook/dpr-ctx_encoder-multiset-base"

# Option 3: Contriever
model_name = "facebook/contriever-msmarco"

embedding_model = HuggingFaceEmbeddings(
    model_name = model_name,
    model_kwargs = {'device': 'cpu'},
    # encode_kwargs
)

/tmp/ipython-input-1404995973.py:10: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/321 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [18]:
db_path = r"G:\My Drive\CMU assign 2\cmu-pittsburgh-rag"
vector_store = Chroma.from_documents(
    documents = data,
    embedding = embedding_model,
    persist_directory = db_path,
)

In [13]:
retriever = vector_store.as_retriever(search_kwargs = {'k'  : 3})

# --- Test your retriever ---
query = "What happened in Pittsburgh in 1845?"
retrieved_docs = retriever.invoke(query)

print(f"\nQuery: {query}")
print("\n--- Retrieved Docs ---")
for i, doc in enumerate(retrieved_docs):
    print(f"--- DOC {i+1} ---")
    print(f"Content: {doc.page_content[:150]}...")
    print(f"Source: {doc.metadata.get('source')}\n")


Query: What happened in Pittsburgh in 1845?

--- Retrieved Docs ---
--- DOC 1 ---
Content: Merthyr Rising
. By the 1840s, Pittsburgh was one of the largest cities west of the
Allegheny Mountains
. The
Great Fire of Pittsburgh
destroyed over ...
Source: https://en.wikipedia.org/wiki/Pittsburgh

--- DOC 2 ---
Content: Merthyr Rising
. By the 1840s, Pittsburgh was one of the largest cities west of the
Allegheny Mountains
. The
Great Fire of Pittsburgh
destroyed over ...
Source: https://en.wikipedia.org/wiki/History_of_Pittsburgh

--- DOC 3 ---
Content: Following the
American Revolution
, the village of Pittsburgh continued to grow. One of its earliest industries was boat building for settlers of the
...
Source: https://en.wikipedia.org/wiki/History_of_Pittsburgh



__Loading database__

In [19]:
from google.colab import drive
drive.mount('/content/drive')

from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

persist_path = r"/content/drive/MyDrive/CMU assign 2/cmu-pittsburgh-rag/G:/My Drive/CMU assign 2/cmu-pittsburgh-rag"
embedding_model = HuggingFaceBgeEmbeddings(
    model_name = "facebook/contriever-msmarco",
    model_kwargs = {'device': 'cpu'}
)

print(f"Loading database from: {persist_path}")
db = Chroma(
    persist_directory = persist_path,
    embedding_function = embedding_model
)

print("✅ Database loaded successfully!")

retriever = db.as_retriever()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/tmp/ipython-input-2276647790.py:9: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceBgeEmbeddings(


Loading database from: G:\My Drive\CMU assign 2\cmu-pittsburgh-rag
✅ Database loaded successfully!


/tmp/ipython-input-2276647790.py:15: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(


In [20]:
# --- Test your retriever ---
query = "What happened in Pittsburgh in 1845?"
retrieved_docs = retriever.invoke(query)

print(f"\nQuery: {query}")
print("\n--- Retrieved Docs ---")
for i, doc in enumerate(retrieved_docs):
    print(f"--- DOC {i+1} ---")
    print(f"Content: {doc.page_content[:150]}...")
    print(f"Source: {doc.metadata.get('source')}\n")


Query: What happened in Pittsburgh in 1845?

--- Retrieved Docs ---
--- DOC 1 ---
Content: Merthyr Rising
. By the 1840s, Pittsburgh was one of the largest cities west of the
Allegheny Mountains
. The
Great Fire of Pittsburgh
destroyed over ...
Source: https://en.wikipedia.org/wiki/Pittsburgh

--- DOC 2 ---
Content: Merthyr Rising
. By the 1840s, Pittsburgh was one of the largest cities west of the
Allegheny Mountains
. The
Great Fire of Pittsburgh
destroyed over ...
Source: https://en.wikipedia.org/wiki/History_of_Pittsburgh

--- DOC 3 ---
Content: Following the
American Revolution
, the village of Pittsburgh continued to grow. One of its earliest industries was boat building for settlers of the
...
Source: https://en.wikipedia.org/wiki/Pittsburgh

--- DOC 4 ---
Content: Following the
American Revolution
, the village of Pittsburgh continued to grow. One of its earliest industries was boat building for settlers of the
...
Source: https://en.wikipedia.org/wiki/History_of_Pittsburgh

